In [2]:
'''
Experiment Description： 使用MRIbydai_minbox（即加入一定像素的边缘），来进行分类
DATA: train data 共600， 每类150； validation data 共233， 分类为
'''
import tensorflow as tf 
import numpy as np 
import os 

batch_size = 20
learning_rate = 1e-3
class_names = ['Her2', 'luminal_A', 'luminal_B', 'TN']#每个分类的名称

In [3]:
#获取data文件路径
data_dir = '/media/ly/liangyi/研究课题——乳腺癌分子分型分类/Data/MRIbydai_minbox/'
Her2_dir = data_dir + 'Her_2/'
luminal_A_dir = data_dir + 'luminal_A/'
luminal_B_dir = data_dir + 'luminal_B/'
TN_dir = data_dir + 'TN/'

In [4]:
#读取每个类别下面图片的数量
Her2_names = [Her2_dir + filename for filename in os.listdir(Her2_dir)]
luminal_A_names = [luminal_A_dir + filename for filename in os.listdir(luminal_A_dir)]
luminal_B_names = [luminal_B_dir + filename for filename in os.listdir(luminal_B_dir)]
TN_names = [TN_dir + filename for filename in os.listdir(TN_dir)]

print(len(Her2_names), len(luminal_A_names), len(luminal_B_names), len(TN_names))

270 179 211 173


In [5]:
#为了保证每类样本的train数量，每类样本训练数量分别提取前150个,共600
Her2_train_names = Her2_names[0:150]
luminal_A_train_names = luminal_A_names[0:150]
luminal_B_train_names = luminal_B_names[0:150]
TN_train_names = TN_names[0:150]

In [6]:
train_names = Her2_train_names + luminal_A_train_names + luminal_B_train_names + TN_train_names
print(len(train_names))

600


In [7]:
#创建train_labels
train_labels = [0] * len(Her2_train_names) + [1] * len(luminal_A_train_names) + [2] * len(luminal_B_train_names) + [3] * len(TN_train_names)
print(len(train_labels))

600


In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_names, train_labels))

In [9]:
def _map(imagename, label):
    image_string = tf.io.read_file(imagename)
    image_decoded = tf.image.decode_png(image_string, channels=1)
    image_resized = tf.image.resize(image_decoded, [40,40])/255.0
    return image_resized, label

In [10]:
train_dataset = train_dataset.map(
    map_func= _map,
    num_parallel_calls= tf.data.experimental.AUTOTUNE
)

In [11]:
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
#检验数据是否倒入正确
print(train_dataset)

<PrefetchDataset shapes: ((None, 40, 40, 1), (None,)), types: (tf.float32, tf.int32)>


In [13]:
#取余下数据集进行测试
Her2_test_names = Her2_names[150:]
luminal_A_test_names = luminal_A_names[150:]
luminal_B_test_names = luminal_B_names[150:]
TN_test_names = TN_names[150:]
test_names = Her2_test_names + luminal_A_test_names + luminal_B_test_names + TN_test_names
print(len(test_names)) 

233


In [14]:
test_labels = [0] * len(Her2_test_names) + [1] * len(luminal_A_test_names) + [2] * len(luminal_B_test_names) + [3] * len(TN_test_names)
print(len(test_labels))

233


In [15]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_names, test_labels))
test_dataset = test_dataset.map(_map).batch(batch_size)
print(test_dataset)

<BatchDataset shapes: ((None, 40, 40, 1), (None,)), types: (tf.float32, tf.int32)>


In [16]:
#搭建网络
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation = tf.nn.relu, input_shape = (40,40,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = tf.nn.relu),
    tf.keras.layers.Dense(64, activation = tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation = tf.nn.sigmoid)
])

In [17]:
num_epochs = 40

In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 128)         147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [20]:
model.fit(train_dataset, validation_data=test_dataset, epochs = num_epochs)

Epoch 1/40
30/30 [==============================] - 0s 12ms/step - loss: 1.3891 - sparse_categorical_accuracy: 0.2250 - val_loss: 1.3874 - val_sparse_categorical_accuracy: 0.1631
Epoch 2/40
30/30 [==============================] - 0s 7ms/step - loss: 1.3848 - sparse_categorical_accuracy: 0.2633 - val_loss: 1.3934 - val_sparse_categorical_accuracy: 0.0987
Epoch 3/40
30/30 [==============================] - 0s 7ms/step - loss: 1.3756 - sparse_categorical_accuracy: 0.2883 - val_loss: 1.3449 - val_sparse_categorical_accuracy: 0.3648
Epoch 4/40
30/30 [==============================] - 0s 7ms/step - loss: 1.3426 - sparse_categorical_accuracy: 0.3283 - val_loss: 1.3376 - val_sparse_categorical_accuracy: 0.1459
Epoch 5/40
30/30 [==============================] - 0s 7ms/step - loss: 1.3149 - sparse_categorical_accuracy: 0.3733 - val_loss: 1.4450 - val_sparse_categorical_accuracy: 0.0987
Epoch 6/40
30/30 [==============================] - 0s 6ms/step - loss: 1.2671 - sparse_categorical_accuracy:

In [21]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation = tf.nn.relu, input_shape = (40,40,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 5, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = tf.nn.relu),
    tf.keras.layers.Dense(4, activation = tf.nn.sigmoid)
])

In [23]:
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
model1.fit(train_dataset, validation_data=test_dataset, epochs = num_epochs)

Epoch 1/40
30/30 [==============================] - 0s 8ms/step - loss: 1.3675 - sparse_categorical_accuracy: 0.3200 - val_loss: 1.4237 - val_sparse_categorical_accuracy: 0.1502
Epoch 2/40
30/30 [==============================] - 0s 5ms/step - loss: 1.3174 - sparse_categorical_accuracy: 0.3750 - val_loss: 1.4376 - val_sparse_categorical_accuracy: 0.2747
Epoch 3/40
30/30 [==============================] - 0s 5ms/step - loss: 1.2148 - sparse_categorical_accuracy: 0.4650 - val_loss: 1.3903 - val_sparse_categorical_accuracy: 0.4292
Epoch 4/40
30/30 [==============================] - 0s 5ms/step - loss: 1.1380 - sparse_categorical_accuracy: 0.5283 - val_loss: 1.6127 - val_sparse_categorical_accuracy: 0.2489
Epoch 5/40
30/30 [==============================] - 0s 5ms/step - loss: 1.1186 - sparse_categorical_accuracy: 0.5417 - val_loss: 1.5146 - val_sparse_categorical_accuracy: 0.3047
Epoch 6/40
30/30 [==============================] - 0s 5ms/step - loss: 0.9717 - sparse_categorical_accuracy: 